In [1]:
# --- shim so "import autoreload" works without changing your code ---
import sys
try:
    import autoreload  # will succeed if already available
except ModuleNotFoundError:
    from IPython.extensions import autoreload as _ar
    sys.modules['autoreload'] = _ar
# -------------------------------------------------------------------

In [2]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler

import autoreload
%reload_ext autoreload
%autoreload 2
import python_code.Scripts as sc
import python_code.Reference as ref

In [3]:
model= pickle.load(open('final_model.sav', 'rb'))

In [4]:
df = pd.read_csv('./data/qc_pub_fy2018.csv')
df.head()

,FSAFIL1,FSAFIL2,FSAFIL3,FSAFIL4,FSAFIL5,FSAFIL6,FSAFIL7,FSAFIL8,FSAFIL9,FSAFIL10,...,WORK12,WORK13,WORK14,WORK15,WORK16,FSNONCIT,FSDIS,FSELDER,FSKID,STATENAME
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,0,1,0,Connecticut
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,1,0,0,Connecticut
2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,1,0,0,Connecticut
3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,Connecticut
4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,1,0,0,Connecticut


In [5]:
df.shape

(43738, 814)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43738 entries, 0 to 43737
Columns: 814 entries, FSAFIL1 to STATENAME
dtypes: float64(373), int64(440), object(1)
memory usage: 271.6+ MB


In [7]:
df = sc.clean_data_to_predict(df)

In [8]:
ss= StandardScaler()
X = ss.fit_transform(df.drop(columns=['state']))
df['predict']=model.predict(X)
df

,state,FSASSET,FSSLTDE2,RAWERND,SHELDED,FSNONCIT,CERTHHSZ,WRK_POOR,FSERNDED,FSNETINC,...,FSNK0T4,FSTOTDED,FSOTHUN,FSUNEARN,FSGA,FSEARN,FSTANF,RAWNET,FSNELDER,predict
0,Connecticut,0.0,584.0,0.0,584.0,0.0,1.0,0.0,0.0,176.0,...,0.0,744.0,0.0,920.0,0.0,0.0,0.0,176.0,1.0,0
1,Connecticut,0.0,969.0,42.0,968.0,0.0,1.0,1.0,42.0,238.0,...,0.0,1171.0,0.0,1197.0,0.0,212.0,0.0,239.0,0.0,0
2,Connecticut,0.0,969.0,0.0,969.0,0.0,1.0,0.0,0.0,149.0,...,0.0,1129.0,0.0,1278.0,0.0,0.0,0.0,149.0,0.0,0
3,Connecticut,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,160.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,Connecticut,0.0,802.0,0.0,802.0,0.0,1.0,0.0,0.0,151.0,...,0.0,962.0,378.0,1113.0,0.0,0.0,0.0,151.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43733,Guam,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,321.0,0.0,150.0,0.0,0.0,0.0,0.0,0.0,0
43734,Guam,0.0,72.0,215.0,342.0,0.0,2.0,1.0,215.0,468.0,...,0.0,608.0,0.0,0.0,0.0,1076.0,0.0,468.0,0.0,0
43735,Guam,0.0,0.0,206.0,124.0,0.0,3.0,1.0,206.0,504.0,...,0.0,527.0,0.0,0.0,0.0,1031.0,0.0,504.0,0.0,0
43736,Guam,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,1.0,321.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [9]:
df.to_csv('./data/2018_indicators/preds_all.csv',index=None)

In [10]:
pred_df = pd.DataFrame(df.groupby('state')['predict'].value_counts().unstack()).rename(columns={0.0:'No',1.0:'Yes'})

In [11]:
pred_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53 entries, Alabama to Wyoming
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   No      53 non-null     int64
 1   Yes     53 non-null     int64
dtypes: int64(2)
memory usage: 1.2+ KB


In [12]:
pred_df.to_csv('./data/2018_indicators/preds_target.csv',index='state')

In [13]:
coef_df = pd.read_csv('./data/2018_indicators/coef.csv')
coef_df['diff']= (coef_df['pred_0']-coef_df['pred_1']).abs()
coef_diff = coef_df.drop(columns=['pred_0','pred_1'])

In [14]:
coef_diff = coef_diff.sort_values(by=['diff'],ascending=False).reset_index(drop=True)
top = coef_diff.head(15)
top.to_csv('./data/2018_indicators/top_impacts.csv',index=None)

In [15]:
coef_2018= sc.top_coef_df(top, df).set_index('state')
coef_2018.astype('int64').to_csv('./data/2018_indicators/coef_2018_all.csv',index='state')

In [16]:
coef_2018.groupby('state').mean().astype('int64').to_csv('./data/2018_indicators/coef_2018_state_avg.csv',index='state')